<a href="https://colab.research.google.com/github/JuanJopz08/AnaliticaNeg/blob/main/ArbolesDecision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. se procede con la carga de las librerias de trabajo

In [ ]:
import pandas as pd
import numpy as np

#para la implementacion y evaluacion del modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Para la carga automatica de los archivos desde el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nxl='/content/6. SolicitantesCrédito(USD) (Original).xlsx'
XDB=pd.read_excel(nxl,sheet_name=1)
XDB.head(100) #solo se muestran 100 datos

XD=XDB[['Edad','Ingresos','Egresos','Monto (EAD)']]
yd=XDB[['PreApr']]

2.Se procede con la implementacion del modelo

* max_depth= Numero de variables disponibles
* gini: es el criterio que indica el grado de error del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)

mar.fit(XD,yd)

#Despliegue del Arbol de Decisión
from sklearn.tree import export_graphviz #Exportar
from pydotplus import graph_from_dot_data #Crear el Gráfico

vs=['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']

dot_data=export_graphviz(mar,feature_names=vs)

graph=graph_from_dot_data(dot_data)
graph.write_png('1.ArbolPreApr.png')


True

In [ ]:
4.Desempeño del modelo

In [ ]:
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0]; FP=cm[0,1]; FN=cm[1,0]; VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN) #funcionamiento general del modelo
sensibilidad=VP/(VP+FN)         #Como se comporto el model con los aprovados
Especificidad=VN/(VN+FP)        #Como se comporto el modelo con los megados
Precision=VP/(VP+FP)            # Cual es el acierto clasificando los positivos
Tasaerror=(FP+FN)/(VP+VN+FP+FN) #Equivocacion general
PreNeg=VN/(VN+FN)               #Como funciona el modelo negando creditos

print('Exactitud:',Exactitud)
print('Sensibilidad:',sensibilidad)
print('Especificidad:',Especificidad)
print('Precision:',Precision)
print('TasaError:',Tasaerror)
print('PreNeg:',PreNeg)

[[2301  658]
 [ 644 2239]]
Exactitud: 0.7771311194796303
Sensibilidad: 0.7766215747485259
Especificidad: 0.7776275768840825
Precision: 0.772868484639282
TasaError: 0.22286888052036974
PreNeg: 0.7813242784380305


5. Creamos el informe para los que no poseen preaprobacion

In [ ]:
#Se toma la totalidad de los solicitante
XDB2=pd.read_excel (nxl, sheet_name=0)
XDB2.head (100)
yd2=XDB2[['PreApr']]
yd2.head(10)

#Donde estan los que no poseen informacion de PreApr
filas=np.where((np.isnan(yd2))==True)[0]
print('Las filas que no poseen informacion:',filas)

#se hace el pronostico de la PreApr para estas personas
XD2=XDB2[['Edad','Ingresos','Egresos','Monto (EAD)']]
XD2=XD2.iloc[filas,]

#Se hace el pronostico para estas personas
ydp2=mar.predict(XD2)
print('Pronostico:',ydp2)

#Crear el informe con la PreApr
dfp=pd.DataFrame(np.column_stack((filas,XD2,ydp2)))
dfp.columns=['filas','Edad','Ingresos','Egresos','Monto (EAD)','PreApr']
dfp.head(10)
dfp.to_excel('InformePreApr.xlsx')

#Cuantos fueron positivos y cuantos negativos
print('los negados pronosticados son:',len(np.where(ydp2==0)[0]))
print('los aprobados pronosticados son:',len(np.where(ydp2==1)[0]))

Las filas que no poseen informacion: [   6    7   11 ... 8367 8372 8377]
Pronostico: [0 0 1 ... 0 1 0]
los negados pronosticados son: 1276
los aprobados pronosticados son: 1260


Analisis de resultados

Para la implementacion del modelo de Arbol de Decisión, se tomaron un total de 5 variables, 4 variables de entrada entre las que se destacan Edad, Ingresos, Egresos y Monto (EAD) (Se debe describir al principio del documento), y una variable de salida que permite evaluar la preaprobación del crédito. Para la construcción del modelo se tuvieron un total de 5842 datos dividos en 2959 créditos Prenegados y un total de 2883 Aprobados.

Luego de la ejecución del modelo se logró una exactitud cercana al 77% en promedio, lo que esta por encima del 75% frente a la exactitud, lo que indica la calidad del modelo en el pronóstico.

Es importante destacar que el coeficiente de sensibilidad estuvo por encima del 75% (77%), mientras que la PredNegativa se ubicó igualmente por encima 75% (78%), lo que corrobora el buen funcionamiento del modelo.

La base de datos completa posee un total de 2536 solicitantes de crédito con ausencia de PreAprobación. Luego de la construcción del modelo se procedio con el pronóstico del valor de preaprobación para esto solicitantes. Luego se logró un informe de preaprobación en donde un total de 1276 (50%) fueron PreNegados y un total de 1260 (50%) fueron Preaprobados.